In [ ]:
import boto3
from dotenv import load_dotenv
load_dotenv()

import os
roleArn = os.getenv("BEDROCKAGENTROLE")
# print(f"Using roleArn: {roleArn}")

Using roleArn: arn:aws:iam::038145523842:role/Agen-access-for-local


In [3]:
bedrock_agent = boto3.client(service_name="bedrock-agent",region_name="us-east-1")

In [6]:
create_agent_response = bedrock_agent.create_agent(
    agentName = "Customer-support-bot-for-e-commerce",
    foundationModel="anthropic.claude-3-haiku-20240307-v1:0",
    instruction = "YOu are an advanced AI agent acting as a frontline xustomer support agent",
    agentResourceRoleArn = roleArn,
)

In [7]:
create_agent_response

{'ResponseMetadata': {'RequestId': '7f278890-857d-4254-8a76-5d318f41bfad',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Mon, 04 Aug 2025 07:58:30 GMT',
   'content-type': 'application/json',
   'content-length': '587',
   'connection': 'keep-alive',
   'x-amzn-requestid': '7f278890-857d-4254-8a76-5d318f41bfad',
   'x-amz-apigw-id': 'OxU2GE1bIAMEmPw=',
   'x-amzn-trace-id': 'Root=1-68906826-1136cd0f5f2cdc65513befb3'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-east-1:038145523842:agent/NNCSQ7FM3I',
  'agentCollaboration': 'DISABLED',
  'agentId': 'NNCSQ7FM3I',
  'agentName': 'Customer-support-bot-for-e-commerce',
  'agentResourceRoleArn': 'arn:aws:iam::038145523842:role/Agen-access-for-local',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2025, 8, 4, 7, 58, 30, 754326, tzinfo=tzutc()),
  'foundationModel': 'anthropic.claude-3-haiku-20240307-v1:0',
  'idleSessionTTLInSeconds': 600,
  'instruction': 'YOu are an advanced AI agent acting as a fr

In [8]:
agentId = create_agent_response['agent']['agentId']


In [9]:
from helper import *

In [11]:
wait_for_agent_status(
    agentId=agentId,
    targetStatus="NOT_PREPARED"
)

Waiting for agent status of 'NOT_PREPARED'...
Agent status: NOT_PREPARED
Agent reached 'NOT_PREPARED' status.


In [12]:
bedrock_agent.prepare_agent(
    agentId=agentId
)

{'ResponseMetadata': {'RequestId': '4bd07450-33af-4d03-b689-8a7f8368101b',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Mon, 04 Aug 2025 08:02:56 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4bd07450-33af-4d03-b689-8a7f8368101b',
   'x-amz-apigw-id': 'OxVfkGJVIAMEJEg=',
   'x-amzn-trace-id': 'Root=1-68906930-2a1f86e716f0d61d14798dca'},
  'RetryAttempts': 0},
 'agentId': 'NNCSQ7FM3I',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2025, 8, 4, 8, 2, 56, 301654, tzinfo=tzutc())}

In [13]:
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='PREPARED'
)

Waiting for agent status of 'PREPARED'...
Agent status: PREPARED
Agent reached 'PREPARED' status.


In [14]:
create_agent_alias_response = bedrock_agent.create_agent_alias(
    agentId=agentId,
    agentAliasName='Aliasformyagent',
)

In [15]:
agentAliasId = create_agent_alias_response['agentAlias']['agentAliasId']

wait_for_agent_alias_status(
    agentId=agentId,
    agentAliasId=agentAliasId,
    targetStatus='PREPARED'
)

Waiting for agent alias status of 'PREPARED'...
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


In [24]:
agentAliasId

'WZZIAZSKHM'

In [16]:
bedrock_agent_runtime = boto3.client(service_name='bedrock-agent-runtime', region_name='us-east-1')

In [17]:
import uuid

In [19]:
message = "My name is abhishek and the iphone is defective and i want a refund"

sessionId = str(uuid.uuid4())

invoke_agent_response = bedrock_agent_runtime.invoke_agent(

    agentId=agentId,
    agentAliasId=agentAliasId, 
    inputText=message,
    sessionId=sessionId,
    endSession=False,
    enableTrace=True,
)

In [20]:
event_stream = invoke_agent_response["completion"]

In [21]:
for event in event_stream:
    print(event)

{'trace': {'agentAliasId': 'WZZIAZSKHM', 'agentId': 'NNCSQ7FM3I', 'agentVersion': '1', 'callerChain': [{'agentAliasArn': 'arn:aws:bedrock:us-east-1:038145523842:agent-alias/NNCSQ7FM3I/WZZIAZSKHM'}], 'eventTime': datetime.datetime(2025, 8, 4, 8, 14, 57, 352297, tzinfo=tzutc()), 'sessionId': '5641a76f-b73c-4a41-a64b-9348075b2294', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'foundationModel': 'anthropic.claude-3-haiku-20240307-v1:0', 'inferenceConfiguration': {'maximumLength': 2048, 'stopSequences': ['</invoke>', '</answer>', '</error>'], 'temperature': 0.0, 'topK': 250, 'topP': 1.0}, 'text': '{"system":" YOu are an advanced AI agent acting as a frontline xustomer support agent You have been provided with a set of functions to answer the user\'s question. You must call the functions in the format below: <function_calls>   <invoke>     <tool_name>$TOOL_NAME</tool_name>     <parameters>       <$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>       ...     </parameters>   </

In [22]:
message = "hey i bought the iphone yesterday and its defectinve i want a refund"

sessionId = str(uuid.uuid4())



In [23]:
invoke_agent_and_print(
    agentAliasId=agentAliasId,
    agentId=agentId,
    sessionId=sessionId,
    inputText=message,
    enableTrace=True,
)

User: hey i bought the iphone yesterday and its defectinve i want a refund

Agent: 
Agent's thought process:
  To help the user get a refund, I will need to gather some more
  information about the issue with the iPhone and the purchase
  details.

Agent's thought process:
  Apologies, let me try that again using the correct function call
  format.

Agent's thought process:
  I apologize, it seems there was an issue with my previous function
  call. Let me try again using the correct format:

Agent's thought process:
  To help the user get a refund, I will need to know more details
  about the issue with the iPhone and the purchase details. I will ask
  the user directly for this information.

Observation:
  Type: FINISH

Final response:
  I apologize, but it seems there is an issue with the function call
  format. Let me try to gather the necessary information in a
  different way:  To help the user get a refund, I will need to know
  more details about the issue with the iPhone and t